# EventVestor: Clinical Trials

In this notebook, we'll take a look at EventVestor's *Clinical Trials* dataset, available on the [Quantopian Store](https://www.quantopian.com/store). This dataset spans January 01, 2007 through the current day, and documents announcements of key phases of clinical trials by biotech/pharmaceutical companies.

### Blaze
Before we dig into the data, we want to tell you about how  you generally access Quantopian Store data sets. These datasets are available through an API service known as [Blaze](http://blaze.pydata.org). Blaze provides the Quantopian user with a convenient interface to access very large datasets.

Blaze provides an important function for accessing these datasets. Some of these sets are many millions of records. Bringing that data directly into Quantopian Research directly just is not viable. So Blaze allows us to provide a simple querying interface and shift the burden over to the server side.

It is common to use Blaze to reduce your dataset in size, convert it over to Pandas and then to use Pandas for further computation, manipulation and visualization.

Helpful links:
* [Query building for Blaze](http://blaze.pydata.org/en/latest/queries.html)
* [Pandas-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-pandas.html)
* [SQL-to-Blaze dictionary](http://blaze.pydata.org/en/latest/rosetta-sql.html).

Once you've limited the size of your Blaze object, you can convert it to a Pandas DataFrames using:
> `from odo import odo`  
> `odo(expr, pandas.DataFrame)`

### Free samples and limits
One other key caveat: we limit the number of results returned from any given expression to 10,000 to protect against runaway memory usage. To be clear, you have access to all the data server side. We are limiting the size of the responses back from Blaze.

There is a *free* version of this dataset as well as a paid one. The free one includes about three years of historical data, though not up to the current day.

With preamble in place, let's get started:

In [2]:
# import the dataset
from quantopian.interactive.data.eventvestor import clinical_trials
# or if you want to import the free dataset, use:
# from quantopian.data.eventvestor import clinical_trials_free

# import data operations
from odo import odo
# import other libraries we will use
import pandas as pd

In [3]:
# Let's use blaze to understand the data a bit using Blaze dshape()
clinical_trials.dshape

dshape("""var * {
  event_id: ?float64,
  asof_date: datetime,
  trade_date: ?datetime,
  symbol: ?string,
  event_type: ?string,
  event_headline: ?string,
  clinical_phase: ?string,
  clinical_scope: ?string,
  clinical_result: ?string,
  product_name: ?string,
  event_rating: ?float64,
  timestamp: datetime,
  sid: ?int64
  }""")

In [4]:
# And how many rows are there?
# N.B. we're using a Blaze function to do this, not len()
clinical_trials.count()

9118

In [5]:
# Let's see what the data looks like. We'll grab the first three rows.
clinical_trials[:3]

,event_id,asof_date,trade_date,symbol,event_type,event_headline,clinical_phase,clinical_scope,clinical_result,product_name,event_rating,timestamp,sid
0,138303,2007-01-03,2007-01-03,IMCL,Clinical Trials,ImClone Systems Commences Patient Treatment in...,Phase I,NaN,NaN,IMC-3G3,1,2007-01-04,3871
1,138180,2007-01-04,2007-01-04,DNA,Clinical Trials,Genentech Announces Positive Results From Rand...,Phase II,NaN,Positive,Pertuzumab,1,2007-01-05,24847
2,952759,2007-01-04,2007-01-04,VICL,Clinical Trials,Vical Initiates Pivotal Phase 3 Trial of Allov...,Phase III,NaN,NaN,Allovectin-7,1,2007-01-05,8763


Let's go over the columns:
- **event_id**: the unique identifier for this clinical trial.
- **asof_date**: EventVestor's timestamp of event capture.
- **trade_date**: for event announcements made before trading ends, trade_date is the same as event_date. For announcements issued after market close, trade_date is next market open day.
- **symbol**: stock ticker symbol of the affected company.
- **event_type**: this should always be *Clinical Trials*.
- **event_headline**: a short description of the event.
- **clinical_phase**: phases include *0, I, II, III, IV, Pre-Clinical*
- **clinical_scope**: types of scope include *additional indications, all indications, limited indications*
- **clinical_result**: result types include *negative, partial, positive*
- **product_name**: name of the drug being investigated.
- **event_rating**: this is always 1. The meaning of this is uncertain.
- **timestamp**: this is our timestamp on when we registered the data.
- **sid**: the equity's unique identifier. Use this instead of the symbol.

We've done much of the data processing for you. Fields like `timestamp` and `sid` are standardized across all our Store Datasets, so the datasets are easy to combine. We have standardized the `sid` across all our equity databases.

We can select columns and rows with ease. Below, we'll fetch all phase-3 announcements. We'll only display the columns for the sid and the drug name.

In [6]:
phase_three = clinical_trials[clinical_trials.clinical_phase == "Phase III"][['timestamp', 'sid','product_name']].sort('timestamp')
# When displaying a Blaze Data Object, the printout is automatically truncated to ten rows.
phase_three

,timestamp,sid,product_name
0,2007-01-05,8763,Allovectin-7
1,2007-01-09,1416,FENTORA
2,2007-01-11,3871,ERBITUX
3,2007-01-25,8763,Allovectin-7
4,2007-02-09,24415,Xibrom
5,2007-02-23,24847,Avastin
6,2007-04-05,3871,ERBITUX (Cetuximab)
7,2007-04-11,3871,ERBITUX
8,2007-04-17,3871,ERBITUX (Cetuximab)
9,2007-04-26,23846,BEMA Fentanyl


Finally, suppose we want a DataFrame of GlaxoSmithKline Phase-III announcements, sorted in descending order by date:

In [7]:
gsk_sid = symbols('GSK').sid

In [8]:
gsk = clinical_trials[clinical_trials.sid==gsk_sid].sort('timestamp',ascending=False)
gsk_df = odo(gsk, pd.DataFrame)
# now filter down to the Phase 4 trials
gsk_df = gsk_df[gsk_df.clinical_phase=="Phase III"]
gsk_df

,event_id,asof_date,trade_date,symbol,event_type,event_headline,clinical_phase,clinical_scope,clinical_result,product_name,event_rating,timestamp,sid
0,1937202,2015-09-27,2015-09-28,GSK,Clinical Trials,GlaxoSmithKline Reports Positive Results From ...,Phase III,NaN,Positive,Anoro Ellipta,1,2015-09-29 11:17:13.121838,3242
3,1836852,2015-02-09,2015-02-09,GSK,Clinical Trials,GlaxoSmithKline and Theravance Initiate Phase ...,Phase III,NaN,NaN,fluticasone furoate/umeclidinium/vilanterol (,1,2015-02-10 00:00:00,3242
4,1817331,2014-12-18,2014-12-18,GSK,Clinical Trials,GlaxoSmithKline Reports ZOE-50 Phase 3 Study M...,Phase III,NaN,Positive,ZOE-50,1,2014-12-19 00:00:00,3242
5,1745987,2014-07-16,2014-07-16,GSK,Clinical Trials,GlaxoSmithKline & Theravance Initiates Phase I...,Phase III,NaN,NaN,IMPACT,1,2014-07-17 00:00:00,3242
6,1738566,2014-06-25,2014-06-25,GSK,Clinical Trials,GlaxoSmithKline Initiates Phase 3 Study with E...,Phase III,NaN,NaN,Eltrombopag,1,2014-06-26 00:00:00,3242
7,1735091,2014-06-13,2014-06-13,GSK,Clinical Trials,GlaxoSmithKline Reports Phase 3 PETIT2 Study M...,Phase III,NaN,Positive,PETIT2,1,2014-06-14 00:00:00,3242
8,1734216,2014-06-11,2014-06-11,GSK,Clinical Trials,GlaxoSmithKline Reports Positive Results from ...,Phase III,NaN,Positive,Incruse Ellipta,1,2014-06-12 00:00:00,3242
10,1707265,2014-04-22,2014-04-22,GSK,Clinical Trials,GlaxoSmithKline and Theravance Starts Phase II...,Phase III,NaN,NaN,FF/VI,1,2014-04-23 00:00:00,3242
11,1700157,2014-04-02,2014-04-02,GSK,Clinical Trials,GlaxoSmithKline to Stop MAGE-A3 Cancer Immunot...,Phase III,NaN,NaN,MAGRITi,1,2014-04-03 00:00:00,3242
12,1695526,2014-03-20,2014-03-20,GSK,Clinical Trials,GlaxoSmithKline's MAGE-A3 Cancer Immunotherape...,Phase III,NaN,Negative,MAGE-A3,1,2014-03-21 00:00:00,3242
